In [21]:
import os
import re
import csv
import json
import random
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

In [22]:
load_dotenv()

client = OpenAI(
    api_key = os.getenv('OPENAI_API_KEY')
)

In [ ]:
# df = pd.read_csv('./path/example_data.csv')
sample_data = pd.read_csv('../train_database/sample.csv')
sample_data_fulltext = [sample_data["text"][9072], sample_data["text"][68], sample_data["text"][3]]
sample_data_summary = [sample_data["summary"][9072], sample_data["summary"][68], sample_data["summary"][3]]
navernews_data = pd.read_csv('../../Non_Finance_data/Naver_Stock/파일이름.csv')

email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
bracket_pattern = r"\[.*?\]"
reporter_pattern = r"\s*[가-힣]+\s*기자\b.*$"

Articles = []

for i in range(len(navernews_data)):
    title = str(navernews_data['Title'][i])
    body = str(navernews_data['Body'][i])

    if re.search(email_pattern, body):
        body = re.split(email_pattern, body)[0]
    
    body = re.sub(bracket_pattern, '', body)
    body = re.sub(reporter_pattern, '', body, flags=re.DOTALL)
    
    Articles.append(f"{title}\n\n {body}")

In [24]:
len(navernews_data)

29951

In [ ]:
duplicate_list = pd.read_csv('../train_database/Summary_gpt.csv')['id'].tolist()
dd = pd.read_csv('../train_database/Summary_groq.csv')['id'].tolist()
duplicate_list.extend(dd)

In [26]:
random.seed(21)
random_idx = random.sample(range(0, len(navernews_data)), 1700)

pseudo_summary_dataset = list(filter(lambda x: len(x[1]) > 30 and x[0] not in duplicate_list, [[i, Articles[i]] for i in random_idx]))

sample = [f"Original Text: {sample_data_fulltext[i]} >> Summary Text: {sample_data_summary[i]}" for i in range(3)]
sample = "\n\n".join(sample)
print(len(pseudo_summary_dataset))

378


In [27]:
prompt = {'role': 'system',
          'content': (
                'You are ChatGPT, an AI language model specializing in economic analysis. '
                'Provide summaries of the news data.'
            )
        }

chat_completion = client.chat.completions.create(
    messages = [
        prompt,
        {
            'role': 'user',
            'content': (
                'Here is the **GOOD** summaries of the news article data:\n\n'
                f'{sample_data}'
                'You MUST provide the summaries based on these **GOOD** summaries within 2 sentences.'
                '이 모든 요약은 한국어로 해야 합니다. 2문장으로 요약해야 합니다.'
            )
        }
    ],
    model = 'gpt-4o',
    temperature = 0.2,
    top_p = 0.5
)

In [ ]:
batch_data = []
csv_file = '../train_database/Summary_gpt.csv'
fieldnames = ['id', 'summary']
for idx in range(0, len(pseudo_summary_dataset)): # -> length 40000: batch 단위 저장 ex... 30개?
    messages = [
        prompt,
        {
            'role': 'user',
            'content': (
                'Summarize the news data by given prompt'
                f'{pseudo_summary_dataset[idx][1]}'
                '이 모든 요약은 한국어로 해야 합니다. 2문장으로 요약해야 합니다.'
            )
        }
    ]
    ncc = client.chat.completions.create(
        messages = messages,
        model = 'gpt-4-turbo',
        temperature=0.2,
        top_p=0.5
    )
    nrc = ncc.choices[0].message.content
    d = {
        'id': pseudo_summary_dataset[idx][0],
        'summary': nrc
    }
    batch_data.append(d)

    if (idx + 1) % 5 == 0:
        # CSV 파일이 이미 존재하는지 확인 (존재하면 헤더 없이 append, 없으면 헤더부터 작성)
        file_exists = os.path.exists(csv_file)
        with open(csv_file, 'a', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            if not file_exists:
                writer.writeheader()
            writer.writerows(batch_data)
        print(f"{idx + 1} records processed so far.")
        batch_data = []

if batch_data:
    file_exists = os.path.exists(csv_file)
    with open(csv_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerows(batch_data)

print("Process ended.")

5 records processed so far.
10 records processed so far.
15 records processed so far.
20 records processed so far.
25 records processed so far.
30 records processed so far.
35 records processed so far.
40 records processed so far.
45 records processed so far.
50 records processed so far.
55 records processed so far.
60 records processed so far.
65 records processed so far.
70 records processed so far.
75 records processed so far.
80 records processed so far.
85 records processed so far.
90 records processed so far.
95 records processed so far.
100 records processed so far.
105 records processed so far.
110 records processed so far.
115 records processed so far.
120 records processed so far.
125 records processed so far.
130 records processed so far.
135 records processed so far.
140 records processed so far.
145 records processed so far.
150 records processed so far.
155 records processed so far.
160 records processed so far.
165 records processed so far.
170 records processed so far.


KeyboardInterrupt: 